# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Estudo de Caso 4</font>
#### <font color='blue'>IA Aplicada ao Direito - Fine-Tuning de Modelo de Linguagem Para Fazer Previsões a Partir de Web Scraping de Textos Jurídicos</font>

## Estudo de Caso 4 - Hardware Necessário Para Executar o Estudo de Caso

- Tentativa 1 - Core i7 com 16 GB de RAM (sem GPU) - Estouro de memória.

- Tentativa 2 - Core i9 com 32 GB de RAM (com GPU Nvidia RTX 3080 de 8 GB de Memória) - Estouro de memória.

- Tentativa 3 - Google Colab Gratuito com 12 GB de RAM (com GPU Tesla T4 de 15 GB de Memória) - Estouro de memória.

- Tentativa 4 - Apple M2 com 96 GB de RAM (sem GPU) - Tempo total de 40 min para executar o treinamento.

- Tentativa 5 - Google Colab Pro com 12 GB de RAM (com GPU V100 de 16 GB de Memória) - Tempo total de pouco mais de 2 min para executar o treinamento.

- Tentativa 6 - Google Colab Pro com 85 GB de RAM (com GPU A100 de 40 GB de Memória) - Tempo total de pouco menos de 2 min para executar o treinamento.

https://en.wikipedia.org/wiki/Ampere_(microarchitecture)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00


In [3]:
!pip install -q torch

In [4]:
!pip install -q transformers==4.32.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.8 MB/s eta 0:00:00


In [5]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [6]:
# Imports
import re
import bs4
import torch
import random
import requests
import transformers
from tqdm import tqdm
from bs4 import BeautifulSoup
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import pipeline

In [7]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

torch       : 2.0.1+cu118
re          : 2.2.1
bs4         : 4.11.2
transformers: 4.32.1
requests    : 2.31.0



## Definindo o Device

In [8]:
# Define o device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [9]:
# Imprime detalhes da GPU (se disponível)
if torch.cuda.is_available():
    print('Número de GPUs:', torch.cuda.device_count())
    print('Modelo da GPU:',torch.cuda.get_device_name(0))
    print('Total de Memória [GB] da GPU:',torch.cuda.get_device_properties(0).total_memory/1e9)

Número de GPUs: 1
Modelo da GPU: NVIDIA A100-SXM4-40GB
Total de Memória [GB] da GPU: 42.481549312


## Verificando Se o Web Site Permite Web Scraping

https://pt.wikipedia.org/robots.txt

https://www.uol.com.br/robots.txt

## Web Scraping de Textos Jurídicos

In [ ]:
# Scraping dos dados com textos jurídicos em português
'''
# Lista
textos_juridicos = []

# Loop
for url in ["https://pt.wikipedia.org/wiki/Direito",
            "https://pt.wikipedia.org/wiki/Constitui%C3%A7%C3%A3o_brasileira_de_1988",
            "https://pt.wikipedia.org/wiki/Embargos_declarat%C3%B3rios",
            "https://pt.wikipedia.org/wiki/Tr%C3%A2nsito_em_julgado",
            "https://pt.wikipedia.org/wiki/Ac%C3%B3rd%C3%A3o"]:

    resp = requests.get(url)

    soup = BeautifulSoup(resp.text, 'html.parser')

    for para in soup.find_all('p'):
        textos_juridicos.append(para.text)
'''

In [10]:
# Scraping dos dados com textos jurídicos em inglês

# Lista
textos_juridicos = []

# Loop
for url in ["https://en.wikipedia.org/wiki/Justice",
            "https://en.wikipedia.org/wiki/Sentence_(law)",
            "https://en.wikipedia.org/wiki/Law",
            "https://en.wikipedia.org/wiki/Federation",
            "https://en.wikipedia.org/wiki/Trial"]:

    resp = requests.get(url)

    soup = BeautifulSoup(resp.text, 'html.parser')

    for para in soup.find_all('p'):
        textos_juridicos.append(para.text)

In [11]:
# Os textos são nossos docs, sendo que cada doc é um parágrafo
docs = textos_juridicos

In [12]:
docs

['\n',
 'Justice, in its broadest sense, is the concept that individuals are to be treated in a manner that is equitable and fair.[1]\n',
 'To achieve justice, individuals should receive that which they deserve, with the interpretation of what "deserve" means, in turn, drawing on numerous viewpoints and perspectives, including fields like ethics, rationality, law, religion, equity and fairness. The state may be said to pursue justice by operating courts and enforcing their rulings.\n',
 'A variety of philosophical and moral theories have been advanced to inform understanding of justice.\n',
 'Early theories of justice were set out by the Ancient Greek philosophers Plato, in his work The Republic, and Aristotle, in his Nicomachean Ethics and Politics. \n',
 'Religious explanations of the justice can be grouped under divine command theory, which holds that justice issues from God.[2]\n',
 'Western thinkers later advanced different theories of where foundations of justice lie.  In the 17t

## Geração dos Arquivos do Texto Extraído via Web Scraping

In [13]:
# Random shuffle
random.shuffle(docs)

In [14]:
# Número de docs (parágrafos)
num_docs = len(docs)
num_docs

221

In [15]:
# Grava o arquivo em disco
with open("dsa_docs.txt", "w") as f:
    f.write("\n".join(docs))

## Modelagem e Tokenização

https://huggingface.co/bert-base-uncased

In [16]:
%env TF_ENABLE_ONEDNN_OPTS=0

env: TF_ENABLE_ONEDNN_OPTS=0


In [17]:
# Carregando modelo e tokenizer em inglês
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# Função de tokenização
def tokenize_function(examples):
    return tokenizer(examples["text"], padding = "max_length", truncation = True)

In [19]:
# Carrega o arquivos de dados de treino
try:
    with open('dsa_docs.txt', 'r') as file:
        data = file.readlines()
    dataset = Dataset.from_dict({'text': data})
except FileNotFoundError:
    print("O arquivo não foi encontrado.")

In [20]:
# Tokeniza o dataset
tokenized_dataset = dataset.map(tokenize_function, batched = True, num_proc = 4, remove_columns = ["text"])

Map (num_proc=4):   0%|          | 0/435 [00:00<?, ? examples/s]

In [21]:
# Formata o dataset
tokenized_dataset.set_format(type = 'torch', columns = ['input_ids', 'attention_mask'])

In [22]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 435
})

In [23]:
# Cria o dataloader
dl_docs = DataLoader(tokenized_dataset, shuffle = True, batch_size = 8)

In [24]:
# Otimizador
optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5)

In [25]:
# Move o modelo para a memória do device
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## Ajustando e Treinando o Modelo

In [26]:
# Coloca o modelo em modo de treino
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [27]:
# Hiperparâmetros
num_epochs = 10
gradient_accumulation_steps = 4

In [28]:
%%time

# Faz um loop através do número de épocas para treinamento
for epoch in range(num_epochs):

    # Enumera cada lote (batch) do dataloader
    for step, batch in enumerate(dl_docs):

        # Move as entradas e as máscaras de atenção para o dispositivo apropriado (CPU ou GPU)
        inputs, attention_mask = batch['input_ids'].to(device), batch['attention_mask'].to(device)

        # Passagem para a frente do modelo para obter as saídas
        outputs = model(inputs, attention_mask = attention_mask, labels = inputs)

        # Extrai a perda (loss) das saídas
        loss = outputs.loss

        # Normaliza a perda ao longo dos passos de acumulação de gradiente
        loss = loss / gradient_accumulation_steps

        # Imprime o valor atual da perda
        print("Erro do Modelo:", loss)

        # Passagem para trás para calcular os gradientes
        loss.backward()

        # Atualiza os pesos do modelo se um número suficiente de passos tiver sido acumulado ou se for o último lote
        if (step + 1) % gradient_accumulation_steps == 0 or step == len(dl_docs) - 1:

            # Realiza o passo de otimização
            optimizer.step()

            # Zera os gradientes
            optimizer.zero_grad()

Erro do Modelo: tensor(4.0803, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(3.6396, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(4.4226, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(4.0074, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(3.3331, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.8605, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.7973, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.8972, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.4098, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.7481, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.5090, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.8824, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.4630, device='cuda:0', grad_fn=<DivBackward0>)
Erro do Modelo: tensor(2.5080, device='cuda:0', grad_fn=<DivBack

In [29]:
# Salva o modelo e o tokenizador
model.save_pretrained("dsa_bert_model")
tokenizer.save_pretrained("dsa_bert_tokenizer")

('dsa_bert_tokenizer/tokenizer_config.json',
 'dsa_bert_tokenizer/special_tokens_map.json',
 'dsa_bert_tokenizer/vocab.txt',
 'dsa_bert_tokenizer/added_tokens.json',
 'dsa_bert_tokenizer/tokenizer.json')

## Previsões com o Modelo Treinado

In [30]:
# Carrega o nosso tokenizador (agora personalizado)
tokenizer_dsa = AutoTokenizer.from_pretrained("dsa_bert_tokenizer")

In [31]:
# Carrega o nosso modelo (agora ajustado)
model_dsa = AutoModelForMaskedLM.from_pretrained("dsa_bert_model")

In [32]:
# Move o modelo para a memória do device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_dsa = model_dsa.to(device)

In [33]:
# Novos dados
texto = "In law, a trial is a coming together of parties to a dispute, to present information in a tribunal"

In [34]:
# Encode do texto (tokenização)
texto_tokenizado = tokenizer_dsa(texto, return_tensors = 'pt')

In [35]:
texto_tokenizado

{'input_ids': tensor([[  101,  1999,  2375,  1010,  1037,  3979,  2003,  1037,  2746,  2362,
          1997,  4243,  2000,  1037,  7593,  1010,  2000,  2556,  2592,  1999,
          1037, 12152,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [36]:
# Coloca os dados na memória do mesmo device do modelo
texto_tokenizado = texto_tokenizado.to(device)

In [37]:
# Previsão
with torch.no_grad():
    output = model(**texto_tokenizado)

In [38]:
# Previsões do modelo
output

MaskedLMOutput(loss=None, logits=tensor([[[ -9.8184, -11.0357, -11.0634,  ...,  -9.4874,  -9.9553,  -7.3561],
         [ -5.7034, -10.2050, -10.6559,  ..., -11.0966, -10.6488, -11.5081],
         [  3.8985,  -2.7178,  -2.7609,  ...,  -4.0518,  -4.0734,  -2.9201],
         ...,
         [ -1.7374,  -6.6276,  -6.7805,  ...,  -6.8806,  -6.0976,  -4.6787],
         [ 10.4552,   3.2917,   2.9061,  ...,   1.7736,  -0.4281,   0.2673],
         [-12.4633, -12.2744, -12.5503,  ...,  -9.7445, -11.6813,  -7.8521]]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [39]:
# Extrai os token ids das previsões (logits de maior valor)
predicted_token_ids = torch.argmax(output.logits, dim = -1)

In [40]:
# Token ids
predicted_token_ids

tensor([[  102,  1999,  2375,  1010,  1037,  3979,  2003,  1037,  2746,  2362,
          1997,  4243,  2000,  1037,  7593,  1010,  2000,  2556,  2592,  1999,
          1037, 12152,   102]], device='cuda:0')

In [41]:
# Decode dos token ids para tokens
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_token_ids[0])

In [42]:
# Tokens previstos
predicted_tokens

['[SEP]',
 'in',
 'law',
 ',',
 'a',
 'trial',
 'is',
 'a',
 'coming',
 'together',
 'of',
 'parties',
 'to',
 'a',
 'dispute',
 ',',
 'to',
 'present',
 'information',
 'in',
 'a',
 'tribunal',
 '[SEP]']

In [43]:
print(predicted_tokens)

['[SEP]', 'in', 'law', ',', 'a', 'trial', 'is', 'a', 'coming', 'together', 'of', 'parties', 'to', 'a', 'dispute', ',', 'to', 'present', 'information', 'in', 'a', 'tribunal', '[SEP]']


## Prevendo Palavras Faltantes em Textos

https://huggingface.co/docs/transformers/main_classes/pipelines

In [44]:
# Novos dados em inglês
# texto = "In law, a trial is a coming together of parties to a dispute, to present information in a tribunal"
texto = "In law, a trial is a coming together of parties to a [MASK], to present information in a tribunal"

In [45]:
# Cria o pipeline
unmasker_pipeline = pipeline('fill-mask', model = model_dsa, tokenizer = tokenizer_dsa, device = 0)

In [46]:
# Prevê o valor da máscara
unmasker_pipeline(texto)

[{'score': 0.44323962926864624,
  'token': 0,
  'token_str': '[PAD]',
  'sequence': 'in law, a trial is a coming together of parties to a, to present information in a tribunal'},
 {'score': 0.0890890508890152,
  'token': 7593,
  'token_str': 'dispute',
  'sequence': 'in law, a trial is a coming together of parties to a dispute, to present information in a tribunal'},
 {'score': 0.037548426538705826,
  'token': 16077,
  'token_str': 'covenant',
  'sequence': 'in law, a trial is a coming together of parties to a covenant, to present information in a tribunal'},
 {'score': 0.018601596355438232,
  'token': 3206,
  'token_str': 'contract',
  'sequence': 'in law, a trial is a coming together of parties to a contract, to present information in a tribunal'},
 {'score': 0.016436513513326645,
  'token': 3979,
  'token_str': 'trial',
  'sequence': 'in law, a trial is a coming together of parties to a trial, to present information in a tribunal'}]

## BERT Pré-Treinado em Português

https://huggingface.co/neuralmind/bert-base-portuguese-cased

# Fim